In [2]:
%load_ext coconut
# Coconut introduces nice functional macros and better map, filter, reduce, etc.
import PyCmdMessenger
from toolz.curried import *
op = operator
from copy import copy, deepcopy
import sys

from typing import *


The coconut extension is already loaded. To reload it, use:
  %reload_ext coconut


In [3]:
# Initialize an ArduinoBoard instance.  This is where you specify baud rate and
# serial timeout.  If you are using a non ATmega328 board, you might also need
# to set the data sizes (bytes for integers, longs, floats, and doubles).  
arduino = PyCmdMessenger.ArduinoBoard("COM3",baud_rate=115200, enable_dtr=False)

# List of command names (and formats for their associated arguments). These must
# be in the same order as in the sketch.
commands = [["MotorOn","ssi"],
            ["MotorOff","s"],
            ["Status","s*"],
            ["kAck","s*"],
            ["kError","s*"],
            ["kLogging", "s*"]]

# Initialize the messenger
c = PyCmdMessenger.CmdMessenger(arduino,commands)


Connecting to arduino on COM3... done.


In [4]:
cmd = PyCmdMessenger
def listen(Messenger: cmd.PyCmdMessenger.CmdMessenger, messageIdentifier) -> Any:
    """ Listens for a specific type of response message"""
    try:
        assert any([messageIdentifier in command for command in Messenger.commands])
        pass
    except:
        raise ValueError("Message identifier must be a valid command identifier for the Messenger")
    while 1:
        message = Messenger.receive()
        if message[0] == messageIdentifier:
            return message
        else:
            continue

def getLogs(Messenger: cmd.PyCmdMessenger.CmdMessenger) -> Text:
    """Yields the logs from the CmdMessenger"""
    while True:
        yield listen(Messenger, "kLogging")
        
def sendCommand(Messenger: cmd.PyCmdMessenger.CmdMessenger, messageIdentifier, *args) -> Any:
    """Sends a command and returns the response"""
    Messenger.send(messageIdentifier, *args)
    if messageIdentifier in [command for command in Messenger.commands]:
        response = listen(Messenger, "kAcknowledge")
    else:
        response = listen(Messenger, "kError")
    return response

In [ ]:
# Send
# c.send("MotorOn","A", "F", 9999)
c.send("Status")
# Receive. Should give ["my_name_is",["Bob"],TIME_RECIEVED]
msg = c.receive(arg_formats="s*")
print(msg)


In [ ]:
msg[1][::2]

In [ ]:
arduino.close()

In [5]:
msg = sendCommand(c, "Status")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
c.send("Status")
[c.receive() for x in range(100)]

In [7]:
arduino.close()